In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

!pip install -qU tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers


In [ ]:
SEED = 42
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)


def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)


set_global_determinism(seed=SEED)

try:
    # TPU config
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    auto = tf.data.experimental.AUTOTUNE
    replicas = strategy.num_replicas_in_sync
    print(f'TPU: {tpu.master()}')
except:
    strategy = tf.distribute.get_strategy()
    auto = tf.data.experimental.AUTOTUNE
    replicas = strategy.num_replicas_in_sync

# XLA acceleartion
tf.config.optimizer.set_jit(True)
print(f'Replicas: {replicas}')
local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")

Replicas: 1


In [ ]:
#tfds.show_examples(ds, ds_info)

In [ ]:
run =   {
    "name": "resnet_model",
    "epochs": 10,
    "iterations": 1,
    "batch_size": 64,
    "learning_rate": 3e-4,
    "class_weights": False,
    "layer_sizes": [
      1024,
      512,
      128
    ],
    "dropout_factor": 0.3
  }

In [ ]:
from sklearn.model_selection import train_test_split
train, info = tfds.load('eurosat/all',
                        with_info=True,
                        split='train',
                        batch_size=-1,
                        data_dir = DATA_DIR
                        )

X = tfds.as_numpy(train['image'])
y = tfds.as_numpy(train['label'])

# Encode labels
y_cat = to_categorical(y, num_classes=10)


X_train, X_test, y_train, y_test = train_test_split(X, y_cat, random_state=0)

In [ ]:
def resnet_model(run, img_input_shape=(500, 500, 3), n_labels=10):
    inputs = layers.Input(shape=img_input_shape)
    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False, weights='imagenet', input_tensor=inputs,
        input_shape=img_input_shape, pooling='avg')
    for i in resnet.layers:
        i.trainable = False
    x = resnet(inputs)
    x = layers.Dense(run['layer_sizes'][0], activation='relu')(x)
    x = layers.Dropout(run['dropout_factor'])(x)
    x = layers.Dense(run['layer_sizes'][1], activation='relu')(x)
    x = layers.Dropout(run['dropout_factor'])(x)
    x = layers.Dense(run['layer_sizes'][2], activation='relu')(x)
    x = layers.Dropout(run['dropout_factor'])(x)

    output = tf.keras.layers.Dense(n_labels, activation='sigmoid', name='output')(x)
    model = tf.keras.Model(inputs=[inputs], outputs=[output])

    print(model.summary())
    return model


In [ ]:
model = resnet_model(run)


In [ ]:
set_seeds(SEED)

tf.keras.backend.clear_session()

img_height = 64
img_width = 64

# data = DataLoader(data_dir, img_height=img_height, img_width=img_width)

# if run['class_weights']:
#     n = data.train_generator.samples
#     n_0 = n - sum(data.train_generator.labels)
#     n_1 = sum(data.train_generator.labels)
#     weight_for_0 = (1 / n_0) * (n / 2.0)
#     weight_for_1 = (1 / n_1) * (n / 2.0)
#     class_weight = {0: weight_for_0, 1: weight_for_1}
# else:
#     class_weight = {0: 1, 1: 1}
if run['name'] == 'resnet_model':
    model = resnet_model(run, img_input_shape=(img_height, img_width, 3))
# elif run_dict['name'] == 'vgg_model':
#     model = vgg_model(run_dict, img_input_shape=(img_height, img_width, 3))
else:
    raise ValueError('Use different model')

loss = tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(learning_rate=run['learning_rate'])
metrics = [tf.keras.metrics.BinaryAccuracy(name='test_accuracy'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf.keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.TruePositives(name='tp')]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)
# log_dir = os.path.join(workdir,
#                         "log_v2/fit/" + run['name'] + f"/{'_'.join([str(x) for x in run.values()])}")

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, min_delta=1e-3,
                                                  restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data = [X_test, y_test], epochs=run['epochs'],
                    callbacks=[early_stopping], verbose=2)


In [ ]:

y_test_flat = np.argmax(y_test, axis=-1)
y_pred = np.argmax(model.predict(X_test), axis=-1)

print(classification_report(y_test_flat, y_pred, target_names=labels))

In [ ]:

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

disp = ConfusionMatrixDisplay.from_predictions(y_test_flat, y_pred,
                              display_labels=labels, ax=ax)
plt.xticks(rotation=45)


# NOTE: Fill all variables here with default values of the plot_confusion_matrix
# disp.plot()

plt.show()